In [1]:
#    Copyright 2020 Arkadip Bhattacharya

#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at

#        http://www.apache.org/licenses/LICENSE-2.0

#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.

In [1]:
import os
import gzip
import requests
import numpy as np
import netCDF4 as nc
from collections import OrderedDict as OD

#### Fetching the Location of a place in `Berlin, Germany`

- Longitude of the location is - 13.3946
- Latitute of the location is - 52.5246

In [2]:
def create_dir(dir_name):
    try:
        os.mkdir(dir_name)
        print(dir_name, "Dir Created")
    except FileExistsError:
        print(dir_name, "Dir Already exists")
    return

def fetch_decompress(url_param, file_name):
    url = 'https://opendata.dwd.de/climate_environment/CDC/grids_germany/monthly/Project_TRY/{}/{}.gz'.format(url_param,file_name)
    with open('./{}/{}'.format(url_param, file_name), 'wb+') as f:
        print('Fetching URL: ', url)
        payload = requests.get(url)
        print("Fetched", file_name)
        dec_payload = gzip.decompress(payload.content)
        print("Decompressed", file_name)
        f.write(dec_payload)
        print('Completed Writing', file_name)
    return

def delete_file(url_param, file_name):
    name = "./{}/{}".format(url_param, file_name)
    
    if os.path.exists(name):
        os.remove(name)
        print("Deleted "+ name)
    else:
        print("The file " + name + " does not exist")
    return

In [3]:
def getclosest_ij(lats,lons,latpt,lonpt):
    # find squared distance of every point on grid
    dist_sq = (lats-latpt)**2 + (lons-lonpt)**2  
    # 1D index of minimum dist_sq element
    minindex_flattened = dist_sq.argmin()    
    # Get 2D index for latvals and lonvals arrays from 1D index
    return np.unravel_index(minindex_flattened, lats.shape)

In [4]:
def get_value(url_param, file_name, latpt, lonpt):
    output_params = {'air_temperature_mean' : 'temperature',
                     'pressure' : 'SLP',
                     'wind_direction' : 'DD',
                     'wind_speed' : 'FF'}
    
    output_variable = output_params[url_param]
    
    with nc.Dataset('./{}/{}'.format(url_param, file_name)) as ncfile:
        print("Opened netCDF: {}".format(file_name))
        lon = ncfile.variables['lon']
        lat = ncfile.variables['lat']
        temp = ncfile.variables[output_variable]
        y, x = getclosest_ij(lat[:], lon[:], latpt, lonpt)
        value = str(temp[0, y, x])
        print("Got the Value {} from {}".format(value, file_name))
        return value

In [5]:
params = OD({'air_temperature_mean' : 'TT',
'pressure' : 'PRED',
'wind_direction' : 'DD',
'wind_speed' : 'FF'})

In [6]:
lon, lat = 13.3946, 52.5246

In [9]:
start_year = 1995
end_year = 2004
start_month = 1
end_month = 12

csvfile = open('./dataset.csv', 'w+')
csvfile.write('time,')
csvfile.write(','.join([i for i in params]))
csvfile.write('\n')

for year in range(start_year, end_year+1):
    for month in range(start_month, end_month+1):
        value_list = []
        value_list.append('{}{:02d}'.format(year, month))
        for url_param, measurement_param in params.items():
            print("Fetching {} of Year: {} of Month: {}".format(url_param, year, month))
            file_name = '{}_{}{:02d}_monmean.nc'.format(measurement_param, year, month)
            create_dir(url_param)
            fetch_decompress(url_param, file_name)
            value = get_value(url_param, file_name, lat, lon)
            value_list.append(str(value))
        value_list.append('\n')
        csv_row = ','.join(value_list)
        print(csv_row+"\n\n")
        csvfile.write(csv_row)
        delete_file(url_param, file_name)

Fetching wind_direction of Year: 1995 of Month: 1
wind_direction Dir Already exists
Fetching URL:  https://opendata.dwd.de/climate_environment/CDC/grids_germany/monthly/Project_TRY/wind_direction/DD_199501_monmean.nc.gz
Fetched DD_199501_monmean.nc
Decompressed DD_199501_monmean.nc
Completed Writing DD_199501_monmean.nc
Opened netCDF: DD_199501_monmean.nc
Got the Value 231.0 from DD_199501_monmean.nc
Fetching air_temperature_mean of Year: 1995 of Month: 1
air_temperature_mean Dir Already exists
Fetching URL:  https://opendata.dwd.de/climate_environment/CDC/grids_germany/monthly/Project_TRY/air_temperature_mean/TT_199501_monmean.nc.gz
Fetched TT_199501_monmean.nc
Decompressed TT_199501_monmean.nc
Completed Writing TT_199501_monmean.nc
Opened netCDF: TT_199501_monmean.nc
Got the Value 1.3000001 from TT_199501_monmean.nc
Fetching wind_speed of Year: 1995 of Month: 1
wind_speed Dir Already exists
Fetching URL:  https://opendata.dwd.de/climate_environment/CDC/grids_germany/monthly/Project_T

In [10]:
csvfile.close()